In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# os.environ['XLA_FLAGS'] ='--xla_gpu_deterministic_ops=true'


os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".5"

# import torch
# torch.multiprocessing.set_start_method('spawn')

In [4]:
# Add parent folder to path (to run this file from subdirectories)
(parent_folder_path, current_dir) = os.path.split(os.path.abspath(''))
sys.path.append(parent_folder_path)
sys.path.append("/homes/80/kang/LOBS5V2/LOBS5/")
sys.path.append("/homes/80/kang/LOBS5/")
sys.path.append("/homes/80/kang/AlphaTrade/")

# add git submodule to path to allow imports to work
submodule_name = 'AlphaTrade'
sys.path.append(os.path.join(parent_folder_path, submodule_name))

print(sys.path)

from gymnax_exchange.jaxob.jorderbook import OrderBook
import gymnax_exchange.jaxob.JaxOrderBookArrays as job

['/data1/sascha/LOBS5/lob', '/data1/sascha/AlphaTrade', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python310.zip', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10/lib-dynload', '', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10/site-packages', '/data1/sascha/LOBS5', '/homes/80/kang/LOBS5V2/LOBS5/', '/homes/80/kang/AlphaTrade/', '/data1/sascha/LOBS5/AlphaTrade']


In [5]:
# from argparse import Namespace
from glob import glob
import numpy as onp
import pandas as pd
# from functools import partial
# from typing import Union, Optional
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
# from line_profiler import LineProfiler

import jax
import jax.numpy as jnp
from jax.nn import one_hot
# from jax import random
# from jax.scipy.linalg import block_diag
# from flax import jax_utils
# from flax.training import checkpoints
# import orbax

#from lob.lob_seq_model import BatchLobPredModel
# from lob.train_helpers import create_train_state, eval_step, prep_batch, cross_entropy_loss, compute_accuracy
from s5.ssm import *
# from s5.ssm_init import make_DPLR_HiPPO
# from s5.dataloading import make_data_loader
# from lob_seq_model import LobPredModel
from encoding import Vocab, Message_Tokenizer
# from lobster_dataloader import LOBSTER_Dataset, LOBSTER_Subset, LOBSTER_Sampler, LOBSTER

import preproc
# import inference
from lob import inference_no_errcorr as inference
import validation_helpers as valh
from lob.init_train import init_train_state, load_checkpoint, load_metadata, load_args_from_checkpoint
# import lob.encoding as encoding

# necessary for flax checkpoints to be loaded in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

from functools import partial



CUDA backend failed to initialize: jaxlib/cuda/versions_helpers.cc:99: operation cuInit(0) failed: CUDA_ERROR_NO_DEVICE.(Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
2024-07-31 13:44:25.198996: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 13:44:25.220004: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 13:44:25.226165: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-31 13:44:26.017143: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Forecasting

In [6]:
import lob.evaluation as eval
from preproc import transform_L2_state

In [7]:
n_gen_msgs = 100  #500 # how many messages to generate into the future
n_messages_conditional = 500
n_eval_messages = 100  # how many to load from dataset 
eval_seq_len = (n_eval_messages-1) * Message_Tokenizer.MSG_LEN

data_levels = 10
# TODO: deprecated - remove from functions
sim_book_levels = 20 # 10  # order book simulator levels
sim_queue_len = 100  # per price in sim, how many orders in queue

n_vol_series = 500  # how many book volume series model uses as input

v = Vocab()
n_classes = len(v)
book_dim = 501 #b_enc.shape[1]
eval_book_seq_len = eval_seq_len


In [8]:
import lob.evaluation as eval
from preproc import transform_L2_state
n_gen_msgs = 100  #500 # how many messages to generate into the future
n_messages = 500
n_eval_messages = 100  # how many to load from dataset 
eval_seq_len = (n_eval_messages-1) * Message_Tokenizer.MSG_LEN

data_levels = 10
# TODO: deprecated - remove from functions
sim_book_levels = 20 # 10  # order book simulator levels
sim_queue_len = 100  # per price in sim, how many orders in queue

n_vol_series = 500  # how many book volume series model uses as input

v = Vocab()
n_classes = len(v)
book_dim = 501 #b_enc.shape[1]
eval_book_seq_len = eval_seq_len

In [12]:


rng = jax.random.key(42)
rng, rng_ = jax.random.split(rng)
# entire test set after training data

stock = 'GOOG'
# data_dir = '/homes/80/kang/LOBS5/GOOG_tiny/'


ckpt_path = '/data1/sascha/data/checkpoints/honest-oath-159_3kn3xbd5/'
args = load_metadata(ckpt_path)
args.dir_name = '/data1/sascha/data/GOOG2017to2019/'

args
# load train state from disk

ds = inference.get_dataset(args.dir_name, n_messages, n_eval_messages)

new_train_state, model_cls = init_train_state(
    args,
    n_classes=n_classes,
    seq_len=eval_seq_len,
    book_dim=book_dim,
    book_seq_len=eval_book_seq_len,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:15                                                                                   │
│                                                                                                  │
│   12 args                                                                                        │
│   13 # load train state from disk                                                                │
│   14                                                                                             │
│ ❱ 15 ds = inference.get_dataset(args.dir_name, n_messages, n_eval_messages)                      │
│   16                                                                                             │
│   17 new_train_state, model_cls = init_train_state(                                              │
│   18 │   args,                                                                                   │
│                                                                                                  │
│ /data1/sascha/LOBS5/lob/inference_no_errcorr.py:196 in get_dataset                               │
│                                                                                                  │
│    193 │   msg_files = sorted(glob(str(data_dir) + '/*message*.npy'))                            │
│    194 │   book_files = sorted(glob(str(data_dir) + '/*book*.npy'))                              │
│    195 │                                                                                         │
│ ❱  196 │   ds = LOBSTER_Dataset(                                                                 │
│    197 │   │   msg_files,                                                                        │
│    198 │   │   n_messages=n_messages + n_eval_messages,                                          │
│    199 │   │   mask_fn=LOBSTER_Dataset.inference_mask,                                           │
│                                                                                                  │
│ /data1/sascha/LOBS5/lob/lobster_dataloader.py:349 in __init__                                    │
│                                                                                                  │
│   346 │   │   │   ) -> None:                                                                     │
│   347 │   │                                                                                      │
│   348 │   │                                                                                      │
│ ❱ 349 │   │   assert len(message_files) > 0                                                      │
│   350 │   │   assert not (use_simple_book and book_transform)                                    │
│   351 │   │                                                                                      │
│   352 │   │   # shift book state by 1 for inference tasks,                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError

In [ ]:
ckpt = load_checkpoint(
    new_train_state,
    ckpt_path,
    train=False,
)
state = ckpt['model']

In [16]:
ds.message_files
import chex
chex.clear_trace_counter()

args = load_metadata(ckpt_path)
batchnorm = args.batchnorm
model = model_cls(training=False, step_rescale=1.0)

In [ ]:
# sample_top_n = 50
# tick_size = 100
# # scale down to single GPU, single sample inference
# args.bsz = 1 #1, 10
# args.num_devices = 1

# CAUTION, SHOULD WE USE THIS ARG?

In [17]:
seq_len = 11000
n_gen_msgs = 200

In [18]:
eval_book_seq_len,  Message_Tokenizer.MSG_LEN, (n_eval_messages-1)

(2178, 22, 99)

In [19]:
# entire test set after training data
stock='GOOG'


if stock == 'GOOG':
    data_dir = '/homes/80/kang/LOBS5V2/LOBS5/GOOG2017to2019'
    ckpt_path='/homes/80/kang/LOBS5/checkpoints/honest-oath-159_3kn3xbd5'
    # data_dir = '/data1/sascha/data/GOOG2017to2019'
    # ckpt_path='/data1/sascha/data/checkpoints/honest-oath-159_3kn3xbd5'
elif stock == 'INTC':
    raise NotImplementedError("Nothing trained for INTC yet")
elif stock == 'TSLA':
    raise Warning("Saved Model was trained on GOOGLE data. Generating for TSLA")
    data_dir = '/data1/sascha/data/lobster_proc'
    ckpt_path = '/data1/sascha/data/checkpoints/honest-oath-159_3kn3xbd5' # Dummy model trained on just 5 days... for debugging. 



In [20]:
args = load_metadata(ckpt_path)
ds = inference.get_dataset(data_dir, n_messages_conditional, n_eval_messages)

new_train_state, model_cls = init_train_state(
    args,
    n_classes=n_classes,
    seq_len=eval_seq_len,
    book_dim=book_dim,
    book_seq_len=eval_book_seq_len,
)
len(ds.message_files)

Dummy input shapes (msg,book) ((Array(8, dtype=int32, weak_type=True), Array(2178, dtype=int32, weak_type=True)), 
 (Array(8, dtype=int32, weak_type=True), Array(2178, dtype=int32, weak_type=True), Array(501, dtype=int32, weak_type=True)))


(12012, 512)
configuring standard optimization setup
[*] Trainable Parameters: 25699641
(12012, 512)
(1, 12012, 512)


In [21]:
import logging
# logging.basicConfig(filename='ar_debug.log', level=logging.DEBUG)
fhandler = logging.FileHandler(filename='generation_debug.log', mode='w')
logger = logging.getLogger()
if (logger.hasHandlers()):
    logger.handlers.clear()
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)
# logger.setLevel(logging.WARNING)

In [22]:
model = model_cls(training=False, step_rescale=1.0)

In [24]:
i = 200
l2_state_n = 20
m_seq, _, b_seq_pv, msg_seq_raw, book_l2_init = ds[int(i)]

b_seq = jnp.array(transform_L2_state(b_seq_pv, n_vol_series, 100))
m_seq_raw_inp = msg_seq_raw[: n_messages_conditional]

# with jax.disable_jit():

# initialise simulator
sim_init, sim_state_init = inference.get_sim(
    book_l2_init,  # book state before any messages
    m_seq_raw_inp, # messages to replay to init sim
)
# # book state after initialisation (replayed messages)
# l2_book_state_init = sim_init.get_L2_state(sim_state_init, l2_state_n)


In [25]:
raw_m_seq = m_seq.copy()
raw_b_seq = b_seq.copy()

In [26]:
sim, sim_state = sim_init, sim_state_init

In [27]:
p_mid = inference._get_safe_mid_price(sim, sim_state, tick_size)

In [28]:
(sim,  # static
train_state,
model,  # static
batchnorm,  # static
encoder,
sample_top_n,  # static
tick_size,  # static
m_seq,
b_seq,
n_msg_todo,  # static
sim_state,
rng) = (sim_init,
new_train_state,
model,
args.batchnorm,
v.ENCODING,
sample_top_n,
tick_size,
m_seq[:11000],
b_seq[:500],
n_gen_msgs,
sim_state_init,
rng)
with jax.ensure_compile_time_eval():
    valid_mask_array = valh.syntax_validation_matrix()

In [ ]:
msg_decoded, sim_state, m_seq, b_seq, book_l2, p_mid, n_msg_todo = inference._generate_msg(
        sim,
        train_state,
        model,
        batchnorm,
        encoder,
        valid_mask_array,
        sample_top_n,
        tick_size,
        m_seq, b_seq, n_msg_todo, p_mid, sim_state, rng_
)

## debug inside the message, targeting the token generation

In [29]:
from datetime import datetime
import functools
from glob import glob
from pathlib import Path
import jax
import jax.numpy as jnp
from jax.nn import one_hot
import flax.linen as nn
from flax.training.train_state import TrainState
from lob import train_helpers
import numpy as onp
import os
import sys
import pandas as pd
import pickle
from functools import partial
from typing import Any, Callable, Dict, Iterable, List, Optional, Sequence, Tuple, Union
from tqdm import tqdm
import logging
logger = logging.getLogger(__name__)
from utils import debug, info

import lob.validation_helpers as valh
import lob.evaluation as eval
import preproc as preproc
from preproc import transform_L2_state
import lob.encoding as encoding
from lob.encoding import Message_Tokenizer, Vocab
from lob.lobster_dataloader import LOBSTER_Dataset

In [30]:
rng, rng_ = jax.random.split(rng)
# treat as compile time constants
with jax.ensure_compile_time_eval():
    l = Message_Tokenizer.MSG_LEN
    last_start_i = m_seq.shape[0] - l

    time_s_start_i, time_s_end_i = valh.get_idx_from_field('time_s')
    time_ns_start_i, time_ns_end_i = valh.get_idx_from_field('time_ns')
    delta_t_s_start_i, delta_t_s_end_i = valh.get_idx_from_field('delta_t_s')
    delta_t_ns_start_i, delta_t_ns_end_i = valh.get_idx_from_field('delta_t_ns')

# 
time_init_s, time_init_ns = encoding.decode_time(
    m_seq[last_start_i + time_s_start_i: last_start_i + time_ns_end_i],
    encoder
)
# roll sequence one step forward
m_seq = valh.append_hid_msg(m_seq)

In [34]:
mask_i = 0
inference._generate_token(
    train_state,
    model,
    batchnorm,
    valid_mask_array,
    sample_top_n,
    m_seq,
    b_seq,
    mask_i,
    rng 
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 mask_i = 0                                                                                  │
│ ❱  2 inference._generate_token(                                                                  │
│    3 │   train_state,                                                                            │
│    4 │   model,                                                                                  │
│    5 │   batchnorm,                                                                              │
│                                                                                                  │
│ /homes/80/kang/LOBS5V2/LOBS5/lob/inference_no_errcorr.py:660 in _generate_token                  │
│                                                                                                  │
│    657 │   │   jnp.ones((1, len(m_seq))),                                                        │
│    658 │   │   jnp.ones((1, len(b_seq)))                                                         │
│    659 │   )                                                                                     │
│ ❱  660 │   logits = valh.predict(                                                                │
│    661 │   │   input,                                                                            │
│    662 │   │   integration_timesteps, train_state, model, batchnorm)                             │
│    663                                                                                           │
│                                                                                                  │
│ /homes/80/kang/LOBS5V2/LOBS5/lob/validation_helpers.py:288 in predict                            │
│                                                                                                  │
│   285 │   │   batchnorm: bool,                                                                   │
│   286 │   ):                                                                                     │
│   287 │   if batchnorm:                                                                          │
│ ❱ 288 │   │   logits = model.apply({"params": state.params, "batch_stats": state.batch_stats},   │
│   289 │   │   │   │   │   │   │   *batch_inputs, *batch_integration_timesteps,                   │
│   290 │   │   │   │   │   │   │   method="__call_rnn__",                                         │
│   291 │   │   │   │   │   │   │   )                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/jax/_src/traceback_util.py:179 in                        │
│ reraise_with_filtered_traceback                                                                  │
│                                                                                                  │
│   176   def reraise_with_filtered_traceback(*args, **kwargs):                                    │
│   177 │   __tracebackhide__ = True                                                               │
│   178 │   try:                                                                                   │
│ ❱ 179 │     return fun(*args, **kwargs)                                                          │
│   180 │   except Exception as e:                                                                 │
│   181 │     mode = _filtering_mode()                                                             │
│   182 │     if _is_under_reraiser(e) or mode == "off":                                           │
│                                                            

# where is the hidden state?

In [31]:
args,args.bsz,args.num_devices

(Namespace(USE_WANDB=True, wandb_project='LOBS5', wandb_entity='sasrey', dir_name='/homes/80/kang/LOBS5/GOOG2017to2019/', dataset='lobster-prediction', masking='none', use_book_data=True, merging='padded', use_simple_book=False, book_transform=True, book_depth=500, restore='/homes/80/kang/LOBS5/checkpoints/rosy-bush-158_pcejove9', restore_step=0, msg_seq_len=500, n_data_workers=8, n_message_layers=2, n_book_pre_layers=1, n_book_post_layers=1, n_layers=12, d_model=512, ssm_size_base=512, blocks=16, C_init='trunc_standard_normal', discretization='zoh', mode='pool', activation_fn='half_glu1', conj_sym=True, clip_eigs=True, bidirectional=False, dt_min=0.001, dt_max=0.1, prenorm=True, batchnorm=True, bn_momentum=0.95, bsz=32, num_devices=4, epochs=100, early_stop_patience=1000, ssm_lr_base=0.0005, lr_factor=1.0, dt_global=False, lr_min=0, cosine_anneal=True, warmup_end=1, lr_patience=1000000, reduce_factor=1.0, p_dropout=0.0, weight_decay=0.05, opt_config='standard', jax_seed=42, debug_load

In [57]:
# hiddens=model_cls().initialize_carry(batch_size=args.bsz, hidden_size=32,
#                          n_message_layers=args.n_message_layers,
#                          n_book_pre_layers=args.n_book_pre_layers ,
#                          n_book_post_layers=args.n_book_post_layers,
#                          n_fused_layers=args.n_layers,)

In [59]:
# len(hiddens),len(hiddens[0]),hiddens[0][0].shape,type(hiddens[0][0]),

(3, 2, (2, 1, 32), jaxlib.xla_extension.ArrayImpl)

In [32]:
hiddens=model_cls().initialize_carry(batch_size=args.bsz//args.num_devices,
                                        hidden_size=(args.ssm_size_base // pow(2,int(args.conj_sym))),
                                        n_message_layers=args.n_message_layers,
                                        n_book_pre_layers=args.n_book_pre_layers ,
                                        n_book_post_layers=args.n_book_post_layers,
                                        n_fused_layers=args.n_layers,)

In [33]:
len(hiddens),len(hiddens[0]),hiddens[0][0].shape,type(hiddens[0][0]),

(3, 2, (8, 1, 256), jaxlib.xla_extension.ArrayImpl)

In [35]:
mask_i = 0

In [36]:
valid_mask = valh.get_valid_mask(valid_mask_array, mask_i)
# input = (
#     jnp.expand_dims(m_seq, axis=0),
#     jnp.expand_dims(b_seq, axis=0)
# )
b_seq = jnp.tile(b_seq, (22, 1))

In [37]:
inputs = (
    m_seq, 
    b_seq,
)
inputs[0].shape,inputs[1].shape


((11000,), (11000, 501))

In [38]:
d_m = jnp.array([False for _ in range(len(m_seq))])
d_b = jnp.array([False for _ in range(len(m_seq))])
d_f = jnp.array([False for _ in range(len(m_seq))])
dones = (
    d_m, d_b, d_f
)
dones[0].shape, dones[1].shape, dones[2].shape

((11000,), (11000,), (11000,))

In [92]:
integration_timesteps = (
    jnp.ones((len(m_seq),)), 
    jnp.ones((len(b_seq),))
)
integration_timesteps[0].shape, integration_timesteps[1].shape

((11000,), (11000,))

In [95]:
hidden_states,logits = model.apply({"params": train_state.params, "batch_stats": train_state.batch_stats},
                    hiddens, *inputs, *dones, *integration_timesteps,
                    method="__call_rnn__"
                    )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 hidden_states,logits = model.apply({"params": train_state.params, "batch_stats": train_s     │
│   2 │   │   │   │   │   hiddens, *inputs, *dones, *integration_timesteps,                        │
│   3 │   │   │   │   │   method="__call_rnn__"                                                    │
│   4 │   │   │   │   │   )                                                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/jax/_src/traceback_util.py:179 in                        │
│ reraise_with_filtered_traceback                                                                  │
│                                                                                                  │
│   176   def reraise_with_filtered_traceback(*args, **kwargs):                                    │
│   177 │   __tracebackhide__ = True                                                               │
│   178 │   try:                                                                                   │
│ ❱ 179 │     return fun(*args, **kwargs)                                                          │
│   180 │   except Exception as e:                                                                 │
│   181 │     mode = _filtering_mode()                                                             │
│   182 │     if _is_under_reraiser(e) or mode == "off":                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/flax/linen/module.py:2252 in apply                       │
│                                                                                                  │
│   2249 │   elif method is None:                                                                  │
│   2250 │     method = self.__call__                                                              │
│   2251 │   method = _get_unbound_fn(method)                                                      │
│ ❱ 2252 │   return apply(                                                                         │
│   2253 │     method,                                                                             │
│   2254 │     self,                                                                               │
│   2255 │     mutable=mutable,                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/flax/core/scope.py:1101 in wrapper                       │
│                                                                                                  │
│   1098 │   with bind(                                                                            │
│   1099 │     variables, rngs=rngs, mutable=mutable, flags=flags                                  │
│   1100 │   ).temporary() as root:                                                                │
│ ❱ 1101 │     y = fn(root, *args, **kwargs)                                                       │
│   1102 │   if mutable is not False:                                                              │
│   1103 │     return y, root.mutable_variables()                                                  │
│   1104 │   else:                                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/flax/linen/module.p

# handle with the dimension of predict_with_hidden

In [97]:
inputs = (
    m_seq[..., None], 
    b_seq,
)
inputs[0].shape,inputs[1].shape

((11000, 1), (11000, 501))

In [43]:
d_m = jnp.array([False for _ in range(len(m_seq))])[..., None]
d_b = jnp.array([False for _ in range(len(m_seq))])[..., None]
d_f = jnp.array([False for _ in range(len(m_seq))])[..., None]
dones = (
    d_m, d_b, d_f
)
dones[0].shape, dones[1].shape, dones[2].shape

((11000, 1), (11000, 1), (11000, 1))

In [109]:
# m_seq = m_seq.squeeze(-1)

In [39]:
m_seq = m_seq[..., None]
m_seq.shape

(11000, 1)

In [40]:
message_integration_timesteps, book_integration_timesteps = jnp.ones((len(m_seq),1)), jnp.ones((len(b_seq),1))
message_integration_timesteps.shape, book_integration_timesteps.shape

((11000, 1), (11000, 1))

In [41]:
integration_timesteps = (
    jnp.ones((len(m_seq),1)), 
    jnp.ones((len(b_seq),1))
)
integration_timesteps[0].shape, integration_timesteps[1].shape

((11000, 1), (11000, 1))

In [44]:
[item.shape for item in (
                            m_seq, b_seq,
                            d_m, d_b,
                            d_f, 
                            message_integration_timesteps, 
                            book_integration_timesteps,)]

[(11000, 1),
 (11000, 501),
 (11000, 1),
 (11000, 1),
 (11000, 1),
 (11000, 1),
 (11000, 1)]

In [45]:
hidden_states,logits = model.apply(
                    {"params": train_state.params, "batch_stats": train_state.batch_stats},
                    hiddens, 
                    m_seq, b_seq,
                    d_m, d_b,
                    d_f, 
                    message_integration_timesteps, 
                    book_integration_timesteps,
                    method="__call_rnn__"
                    )

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 hidden_states,logits = model.apply(                                                         │
│    2 │   │   │   │   │   {"params": train_state.params, "batch_stats": train_state.batch_stat    │
│    3 │   │   │   │   │   hiddens,                                                                │
│    4 │   │   │   │   │   m_seq, b_seq,                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/jax/_src/traceback_util.py:179 in                        │
│ reraise_with_filtered_traceback                                                                  │
│                                                                                                  │
│   176   def reraise_with_filtered_traceback(*args, **kwargs):                                    │
│   177 │   __tracebackhide__ = True                                                               │
│   178 │   try:                                                                                   │
│ ❱ 179 │     return fun(*args, **kwargs)                                                          │
│   180 │   except Exception as e:                                                                 │
│   181 │     mode = _filtering_mode()                                                             │
│   182 │     if _is_under_reraiser(e) or mode == "off":                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/flax/linen/module.py:2252 in apply                       │
│                                                                                                  │
│   2249 │   elif method is None:                                                                  │
│   2250 │     method = self.__call__                                                              │
│   2251 │   method = _get_unbound_fn(method)                                                      │
│ ❱ 2252 │   return apply(                                                                         │
│   2253 │     method,                                                                             │
│   2254 │     self,                                                                               │
│   2255 │     mutable=mutable,                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/flax/core/scope.py:1101 in wrapper                       │
│                                                                                                  │
│   1098 │   with bind(                                                                            │
│   1099 │     variables, rngs=rngs, mutable=mutable, flags=flags                                  │
│   1100 │   ).temporary() as root:                                                                │
│ ❱ 1101 │     y = fn(root, *args, **kwargs)                                                       │
│   1102 │   if mutable is not False:                                                              │
│   1103 │     return y, root.mutable_variables()                                                  │
│   1104 │   else:                                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/flax/linen/module.p

## predict_with_hidden_v2

In [67]:
import flax
@partial(jax.jit, static_argnums=(3, 4))
def predict_with_hidden_v2(
        hidden_states,
        batch_inputs: jax.Array,
        batch_dones, 
        batch_integration_timesteps: jax.Array,
        state: TrainState,
        model: flax.linen.Module,
        batchnorm: bool,
    ):
    batch_inputs
    batch_dones
    
    if batchnorm:
        hidden_states,logits = model.apply({"params": state.params, "batch_stats": state.batch_stats},
                            hidden_states,*batch_inputs, *batch_integration_timesteps,
                            method="__call_rnn__"
                            )
    else:
        hidden_states,logits = model.apply({"params": state.params},
                             hidden_states,*batch_inputs, *batch_integration_timesteps,
                             method="__call_rnn__"
                             )

    return hidden_states,logits

# test generate token v2

In [ ]:
def _generate_token_v2(
        train_state : TrainState,
        model : nn.module,
        batchnorm : bool,
        valid_mask_array : jax.Array ,
        sample_top_n : int,
        hidden_state: Tuple , #Assume 'something', probably model params. 
        tok: jax.Array, #Should be needed only for 1st token.
        book: jax.Array , #Should only need for 1st token.
        mask_i : int, #Still between 0 and message_length
        rng 
    ):

    # syntactically valid tokens for current message position
    valid_mask = valh.get_valid_mask(valid_mask_array, mask_i)

    b_seq = jnp.tile(b_seq, (22, 1)) # NOTE only used for debugging
    inputs = (
        m_seq[..., None], 
        b_seq,
    )
    integration_timesteps = (
        jnp.ones((len(m_seq),1)), 
        jnp.ones((len(b_seq),1))
    )
    d_m = jnp.array([False for _ in range(len(m_seq))])[..., None]
    d_b = jnp.array([False for _ in range(len(m_seq))])[..., None]
    d_f = jnp.array([False for _ in range(len(m_seq))])[..., None]
    dones = (
        d_m, d_b, d_f
    )

    hidden,logits = valh.predict_with_hidden_v2(
        hidden,inputs,dones,
        integration_timesteps, train_state, model, batchnorm)
    
    # filter out (syntactically) invalid tokens for current position
    if valid_mask is not None:
        logits = valh.filter_valid_pred(logits, valid_mask)

    # update sequence
    # NOTE: rng arg expects one element per batch element
    rng, rng_ = jax.random.split(rng)
    
    # ATTEMPT 1
    # m_seq = valh.fill_predicted_toks(m_seq, logits, sample_top_n, jnp.array([rng_]))
    # new_tok = m_seq[mask_i] # CAUTION not sure if this is correct
    
    # ATTEMPT 2
    new_tok = valh.sample_pred(logits, sample_top_n, jnp.array([rng_]))
    
    # ATTEMPT 3
    # new_tok = valh.pred_next_tok(
    #                     seq,
    #                     state,
    #                     model,
    #                     batchnorm,
    #                     sample_top_n,
    #                     mask_i,
    #                     rng,
    #                     vocab_len,
    #                     book_seq=None,
    #                     new_msg=False,
    #                     valid_mask=None,  
    #                 )
                    
    return new_tok,mask_i + 1, rng,hidden

In [ ]:
def _generate_token(
        train_state : TrainState,
        model : nn.module,
        batchnorm : bool,
        valid_mask_array : jax.Array ,
        sample_top_n : int,
        m_seq: jax.Array ,
        b_seq: jax.Array ,
        mask_i : int,
        rng 
    ):

    # syntactically valid tokens for current message position
    valid_mask = valh.get_valid_mask(valid_mask_array, mask_i)
    m_seq, _ = valh.mask_last_msg_in_seq(m_seq, mask_i)
    input = (
        jnp.expand_dims(m_seq, axis=0),
        jnp.expand_dims(b_seq, axis=0)
    )
    integration_timesteps = (
        jnp.ones((1, len(m_seq))), 
        jnp.ones((1, len(b_seq)))
    )
    logits = valh.predict(
        input,
        integration_timesteps, train_state, model, batchnorm)
    
    # filter out (syntactically) invalid tokens for current position
    if valid_mask is not None:
        logits = valh.filter_valid_pred(logits, valid_mask)

    # update sequence
    # NOTE: rng arg expects one element per batch element
    rng, rng_ = jax.random.split(rng)
    m_seq = valh.fill_predicted_toks(m_seq, logits, sample_top_n, jnp.array([rng_]))

    return m_seq, mask_i + 1, rng

# _make_generate_token_scannable v2

In [ ]:
def _make_generate_token_scannable_v2(
        train_state: TrainState,
        model: nn.Module,
        batchnorm: bool,
        valid_mask_array: jax.Array,
        sample_top_n: int,
    ):
    """
    Create a scannable function for token generation using _generate_token_v2.
    """
    __generate_token_v2 = jax.jit(functools.partial(
        _generate_token_v2, train_state, model, batchnorm, valid_mask_array, sample_top_n
    ))

    def _generate_token_scannable(carry, xs):
        m_seq, b_seq, mask_i, rng, hidden_state = carry
        
        # Extract the last token and book state
        tok = m_seq[mask_i - 1] if mask_i > 0 else m_seq[0]
        book = b_seq[-1]

        new_tok, mask_i, rng, hidden_state = __generate_token_v2(
            hidden_state, tok, book, mask_i, rng
        )
        
        # Update m_seq with the new token
        m_seq = m_seq.at[mask_i - 1].set(new_tok)

        return (m_seq, b_seq, mask_i, rng, hidden_state), new_tok

    return _generate_token_scannable

# Update the usage of generate_token_scannable
generate_token_scannable_v2 = _make_generate_token_scannable_v2(
    train_state, model, batchnorm, valid_mask_array, sample_top_n
)

# get next message: generate l tokens:
# generate tokens until time is reached
mask_i = 0
hidden_state = model.init_hidden()  # You need to implement this method in your model
gen_token_carry = (m_seq, b_seq, mask_i, rng_, hidden_state)
(m_seq, b_seq, mask_i, rng_, hidden_state), new_toks = jax.lax.scan(
    generate_token_scannable_v2,
    gen_token_carry,
    xs=None,
    length=TIME_START_I
)

In [ ]:
generate_token_scannable = _make_generate_token_scannable(
    train_state, model, batchnorm, valid_mask_array, sample_top_n
)
# get next message: generate l tokens:
# generate tokens until time is reached
mask_i = 0
gen_token_carry = (m_seq, b_seq, mask_i, rng_)
(m_seq, b_seq, mask_i, rng_), _ = jax.lax.scan(
    generate_token_scannable,
    gen_token_carry,
    xs=None,
    length=TIME_START_I
)

In [ ]:
def _make_generate_token_scannable(
        train_state: TrainState,
        model: nn.Module,
        batchnorm: bool,
        valid_mask_array: jax.Array,
        sample_top_n: int,
    ):
    """
    """
    __generate_token = jax.jit(functools.partial(
        _generate_token, train_state, model, batchnorm, valid_mask_array, sample_top_n
    ))

    def _generate_token_scannable(carry, xs):
        # m_seq, b_seq, mask_i, rng = carry
        m_seq, mask_i, rng = __generate_token(*carry)
        return (m_seq, carry[1], mask_i, rng), None

    return _generate_token_scannable

# _generate_msg_v2

In [ ]:
def _generate_msg_v2(
        sim: OrderBook,
        train_state: TrainState,
        model: nn.Module,
        batchnorm: bool,
        encoder: Dict[str, Tuple[jax.Array, jax.Array]],
        valid_mask_array: jax.Array,
        sample_top_n: int,
        tick_size: int,
        
        m_seq: jax.Array,
        b_seq: jax.Array,
        n_msg_todo: int,
        p_mid: jax.Array,
        sim_state: LobState,
        rng: jax.dtypes.prng_key,
    ) -> Tuple[jax.Array, LobState, jax.Array, jax.Array, jax.Array, jax.Array, int, Any]:
    """
    Generate a single message using generate_token_scannable_v2
    """
    rng, rng_ = jax.random.split(rng)
    # treat as compile time constants
    with jax.ensure_compile_time_eval():
        l = Message_Tokenizer.MSG_LEN
        last_start_i = m_seq.shape[0] - l

        time_s_start_i, time_s_end_i = valh.get_idx_from_field('time_s')
        time_ns_start_i, time_ns_end_i = valh.get_idx_from_field('time_ns')
        delta_t_s_start_i, delta_t_s_end_i = valh.get_idx_from_field('delta_t_s')
        delta_t_ns_start_i, delta_t_ns_end_i = valh.get_idx_from_field('delta_t_ns')

    # 
    time_init_s, time_init_ns = encoding.decode_time(
        m_seq[last_start_i + time_s_start_i: last_start_i + time_ns_end_i],
        encoder
    )

    # roll sequence one step forward
    m_seq = valh.append_hid_msg(m_seq)
    first_message = m_seq[:Message_Tokenizer.MSG_LEN]

    # Initialize hidden state
    hidden_state = model.init_hidden()

    # get next message: generate l tokens:
    # generate tokens until time is reached
    mask_i = 0
    gen_token_carry = (m_seq, b_seq, mask_i, rng_, hidden_state)
    (m_seq, b_seq, mask_i, rng_, hidden_state), _ = jax.lax.scan(
        generate_token_scannable_v2,
        gen_token_carry,
        xs=None,
        length=TIME_START_I
    )
    # fill the time tokens
    m_seq = _add_time_tokens(
        m_seq,
        encoder,
        time_init_s,
        time_init_ns,
        last_start_i,
        time_s_start_i,
        time_ns_end_i,
        delta_t_s_start_i,
        delta_t_s_end_i,
        delta_t_ns_start_i,
        delta_t_ns_end_i,
    )
    # update mask index to skip time token positions
    mask_i = TIME_END_I
    gen_token_carry = (m_seq, b_seq, mask_i, rng_, hidden_state)

    # finish message generation
    (m_seq, b_seq, mask_i, rng_, hidden_state), _ = jax.lax.scan(
        generate_token_scannable_v2,
        gen_token_carry,
        xs=None,
        length=l-TIME_END_I
    )

    order_id = n_msg_todo

    sim_msg, msg_decoded = get_sim_msg(
        m_seq[-l:],  # the generated message
        sim,
        sim_state,
        mid_price = p_mid,
        new_order_id = order_id,
        tick_size = tick_size,
        encoder = encoder,
    )

    # feed message to simulator, updating book state
    sim_state = sim.process_order_array(sim_state, sim_msg)

    # get current mid price from simulator
    p_mid_new = _get_new_mid_price(sim, sim_state, p_mid, tick_size)

    # price change in ticks
    p_change = ((p_mid_new - p_mid) // tick_size)

    # get new book state
    book_l2 = sim.get_L2_state(sim_state, l2_state_n)

    new_book_raw = jnp.concatenate([jnp.array([p_change]), book_l2]).reshape(1,-1)
    new_book = preproc.transform_L2_state(new_book_raw, 500, 100)
    # update book sequence
    b_seq = jnp.concatenate([b_seq[1:], new_book])

    n_msg_todo -= 1

    return msg_decoded, sim_state, m_seq, b_seq, book_l2, p_mid_new, n_msg_todo, hidden_state

In [ ]:
def _generate_msg(
        sim: OrderBook,
        train_state: TrainState,
        model: nn.Module,
        batchnorm: bool,
        encoder: Dict[str, Tuple[jax.Array, jax.Array]],
        valid_mask_array: jax.Array,
        sample_top_n: int,
        tick_size: int,
        
        m_seq: jax.Array,
        b_seq: jax.Array,
        n_msg_todo: int,
        p_mid: jax.Array,
        sim_state: LobState,
        rng: jax.dtypes.prng_key,
    ) -> Tuple[jax.Array, LobState, jax.Array, jax.Array, jax.Array, jax.Array, int]:
    """
    """
    rng, rng_ = jax.random.split(rng)
    # treat as compile time constants
    with jax.ensure_compile_time_eval():
        l = Message_Tokenizer.MSG_LEN
        last_start_i = m_seq.shape[0] - l

        time_s_start_i, time_s_end_i = valh.get_idx_from_field('time_s')
        time_ns_start_i, time_ns_end_i = valh.get_idx_from_field('time_ns')
        delta_t_s_start_i, delta_t_s_end_i = valh.get_idx_from_field('delta_t_s')
        delta_t_ns_start_i, delta_t_ns_end_i = valh.get_idx_from_field('delta_t_ns')

    # 
    time_init_s, time_init_ns = encoding.decode_time(
        m_seq[last_start_i + time_s_start_i: last_start_i + time_ns_end_i],
        encoder
    )

    # roll sequence one step forward
    m_seq = valh.append_hid_msg(m_seq)

    # TODO: calculating time in case where generation is not sequentially left to right
    #       --> check if delta_t complete --> calc time once

    generate_token_scannable = _make_generate_token_scannable(
        train_state, model, batchnorm, valid_mask_array, sample_top_n
    )

    # get next message: generate l tokens:
    # generate tokens until time is reached
    mask_i = 0
    gen_token_carry = (m_seq, b_seq, mask_i, rng_)
    (m_seq, b_seq, mask_i, rng_), _ = jax.lax.scan(
        generate_token_scannable,
        gen_token_carry,
        xs=None,
        length=TIME_START_I
    )
    # fill the time tokens
    m_seq = _add_time_tokens(
        m_seq,
        encoder,
        time_init_s,
        time_init_ns,
        last_start_i,
        time_s_start_i,
        time_ns_end_i,
        delta_t_s_start_i,
        delta_t_s_end_i,
        delta_t_ns_start_i,
        delta_t_ns_end_i,
    )
    # update mask index to skip time token positions
    mask_i = TIME_END_I
    gen_token_carry = (m_seq, b_seq, mask_i, rng_)

    # finish message generation
    (m_seq, b_seq, mask_i, rng_), _ = jax.lax.scan(
        generate_token_scannable,
        gen_token_carry,
        xs=None,
        length=l-TIME_END_I
    )

    # order_id = id_gen.step()  # no order ID generator any more in v3 sim?
    order_id = n_msg_todo

    sim_msg, msg_decoded = get_sim_msg(
        m_seq[-l:],  # the generated message
        # m_seq[:-l],  # sequence without generated message
        # m_seq_raw[1:],   # raw data (same length as sequence without generated message)
        # None,
        sim,
        sim_state,
        mid_price = p_mid,
        new_order_id = order_id,
        tick_size = tick_size,
        encoder = encoder,
    )

    # jax.debug.print('sim_msg {}', sim_msg)

    # feed message to simulator, updating book state
    sim_state = sim.process_order_array(sim_state, sim_msg)

    # debug('trades', _trades)

    # get current mid price from simulator
    p_mid_new = _get_new_mid_price(sim, sim_state, p_mid, tick_size)
    # jax.debug.print('p_mid_new {}', p_mid_new)

    # price change in ticks
    p_change = ((p_mid_new - p_mid) // tick_size)#.astype(jnp.int32)

    # get new book state
    book_l2 = sim.get_L2_state(sim_state, l2_state_n)
    # l2_book_states.append(book_l2)

    # error if the new message does not change the book state
    # is_error = (book_l2 == b_seq[-1, 1:]).all()

    new_book_raw = jnp.concatenate([jnp.array([p_change]), book_l2]).reshape(1,-1)
    new_book = preproc.transform_L2_state(new_book_raw, 500, 100)
    # update book sequence
    b_seq = jnp.concatenate([b_seq[1:], new_book])

    n_msg_todo -= 1

    return msg_decoded, sim_state, m_seq, b_seq, book_l2, p_mid_new, n_msg_todo

    

In [ ]:
@partial(jax.jit, static_argnums=(0, 2, 3, 5, 6, 9))
def generate(
        sim: OrderBook,  # static
        train_state: TrainState,
        model: nn.Module,  # static
        batchnorm: bool,  # static
        encoder: Dict[str, Tuple[jax.Array, jax.Array]],
        sample_top_n: int,  # static
        tick_size: int,  # static
        m_seq: jax.Array,
        b_seq: jax.Array,
        n_msg_todo: int,  # static
        sim_state: LobState,
        rng: jax.dtypes.prng_key,
        # if eval_msgs given, also returns loss of predictions
        # e.g. to calculate perplexity
        # m_seq_eval: Optional[jax.Array] = None,  
    ) -> Tuple[jax.Array, jax.Array, jax.Array, jax.Array, jax.Array]:

In [ ]:
m_seq, b_seq, msgs_decoded, l2_book_states, num_errors = inference.generate(
    sim_init,
    new_train_state,
    model,
    args.batchnorm,
    v.ENCODING,
    sample_top_n,
    tick_size,
    m_seq[:11000],
    b_seq[:500],
    n_gen_msgs,
    sim_state_init,
    rng,
)

# Essentials

In [ ]:
from functools import partial

@partial(jax.jit, static_argnames=['n'])
def g(x, n):
  for i in range(n):
    x = x ** 2
  return x

g(jnp.arange(4), 3)

Array([   0,    1,  256, 6561], dtype=int32)

In [ ]:
from functools import partial

# @partial(jax.jit, static_argnames=['n'])
@jax.jit
def g(x, n):
  for i in range(n):
    x = x ** 2
  return x
g(jnp.arange(4), 3)

Array([   0,    1,  256, 6561], dtype=int32)

In [ ]:
integration_timesteps = (
    jnp.ones((1, len(m_seq))), 
    jnp.ones((1, len(b_seq)))
)
integration_timesteps[0].shape, integration_timesteps[1].shape